In [50]:
import joblib
import os
import pandas as pd

In [51]:
# # Chuẩn bị dữ liệu mới
# new_data = pd.DataFrame([[
#     'Oral', 'Below Normal', 'Above Normal', 150, 'Below Normal', 'Venturi Mask',
#     26.8352, 'Normal', 'Below Normal', 'Not tested', 'Not tested', 'Normal', 
#     'Normal', 'Not tested', 'Above Normal', 'Below Normal', 'Below Normal', 
#     'Normal', 'Above Normal', 'Above Normal', True, 'Not applied', 'Applied',
#     'Above Normal', 'Below Normal', 'Above Normal'
# ]], columns=[
#     'NutritionIntake', 'Cortisol', 'ProBNP', 'UrineOutput', 'K', 'OxygenTherapy', 
#     'BMI', 'Pulse', 'eGFR', 'TSH', 'FT3', 'Cl', 'Na', 'FT4', 'CRP', 'Ca', 'Mg', 
#     'Ure', 'PCT', 'Lactat', 'Antibiotics', 'Adrenalin', 'Noradrenalin', 'Creatinin', 
#     'PaCO2', 'TroponinI'
# ])

# # Mã hóa các thuộc tính phân loại
# categorical_cols = ['NutritionIntake', 'Cortisol', 'ProBNP', 'K', 'OxygenTherapy', 
#                     'Pulse', 'eGFR', 'TSH', 'FT3', 'Cl', 'Na', 'FT4', 'CRP', 
#                     'Ca', 'Mg', 'Ure', 'PCT', 'Lactat', 'Adrenalin', 
#                     'Noradrenalin', 'Creatinin', 'PaCO2', 'TroponinI']

# for col in categorical_cols:
#     if col in new_data.columns:
#         new_data[col] = label_encoders[col].transform(new_data[col])

# # Dự đoán với dữ liệu mới
# predictions = model.predict(new_data)

# print(predictions)

In [52]:
data = pd.read_excel("./data/BME10.xlsx")

In [53]:
data.head()

,Unnamed: 0,PatientID,Age,Gender,BMI,MedicalConditions_InfectiousStatus,SurgicalConditions,UMC,Pulse_admission,RespiratoryRate_admission,...,OxygenTherapy_admission,Antibiotics_admission,Adrenalin_admission,Noradrenalin_admission,Dobutamin_admission,Dopamin_admission,NutritionIntake_admission,FluidIntake_admission,UrineOutput_admission,Survival
0,0,20035848,83,Female,18.365473,Infected,No surgery,True,Above Normal,Normal,...,Non-Invasive Ventilation,True,Not applied,Not applied,Applied,Not applied,Nasogastric Tube,455,41.6,Death
1,1,22167297,64,Female,24.218750,Infected,No surgery,True,Above Normal,Above Normal,...,Mechanical Ventilation,False,Not applied,Applied,Not applied,Not applied,Nasogastric Tube,1917,471.0,Alive
2,2,16005539,73,Female,21.367521,Infected,No surgery,False,Normal,Above Normal,...,Nasal Cannula,False,Not applied,Not applied,Not applied,Not applied,Intravenous,1332,170.0,Death
3,3,17007469,72,Female,26.835180,Infected,No surgery,True,Normal,Normal,...,Venturi Mask,True,Not applied,Applied,Not applied,Not applied,Oral,1010,150.0,Alive
4,4,22904879,57,Male,18.365473,Infected,No surgery,True,Above Normal,Above Normal,...,HFNC,True,Not applied,Not applied,Not applied,Not applied,Nasogastric Tube,1693,90.0,Death


In [54]:
def remove_suffix(df: pd.DataFrame, suffix: str) -> pd.DataFrame:
    """
    Rename columns by removing a specified suffix from the column names.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    suffix (str): The suffix to remove from the column names.

    Returns:
    pd.DataFrame: DataFrame with renamed columns.
    """
    new_columns = {}

    for col in df.columns:
        if col.endswith(suffix):
            new_col = col[:-len(suffix)]
            new_columns[col] = new_col
        else:
            new_columns[col] = col

    df.rename(columns=new_columns, inplace=True)

    return df

data = remove_suffix(data, '_admission')

In [55]:
data.head()

,Unnamed: 0,PatientID,Age,Gender,BMI,MedicalConditions_InfectiousStatus,SurgicalConditions,UMC,Pulse,RespiratoryRate,...,OxygenTherapy,Antibiotics,Adrenalin,Noradrenalin,Dobutamin,Dopamin,NutritionIntake,FluidIntake,UrineOutput,Survival
0,0,20035848,83,Female,18.365473,Infected,No surgery,True,Above Normal,Normal,...,Non-Invasive Ventilation,True,Not applied,Not applied,Applied,Not applied,Nasogastric Tube,455,41.6,Death
1,1,22167297,64,Female,24.218750,Infected,No surgery,True,Above Normal,Above Normal,...,Mechanical Ventilation,False,Not applied,Applied,Not applied,Not applied,Nasogastric Tube,1917,471.0,Alive
2,2,16005539,73,Female,21.367521,Infected,No surgery,False,Normal,Above Normal,...,Nasal Cannula,False,Not applied,Not applied,Not applied,Not applied,Intravenous,1332,170.0,Death
3,3,17007469,72,Female,26.835180,Infected,No surgery,True,Normal,Normal,...,Venturi Mask,True,Not applied,Applied,Not applied,Not applied,Oral,1010,150.0,Alive
4,4,22904879,57,Male,18.365473,Infected,No surgery,True,Above Normal,Above Normal,...,HFNC,True,Not applied,Not applied,Not applied,Not applied,Nasogastric Tube,1693,90.0,Death


In [56]:
matching_rows_diag = data.loc[data['PatientID'].astype(str) == str(19014739)]

In [57]:
matching_rows_diag

,Unnamed: 0,PatientID,Age,Gender,BMI,MedicalConditions_InfectiousStatus,SurgicalConditions,UMC,Pulse,RespiratoryRate,...,OxygenTherapy,Antibiotics,Adrenalin,Noradrenalin,Dobutamin,Dopamin,NutritionIntake,FluidIntake,UrineOutput,Survival
22,22,19014739,64,Male,27.34375,Infected,No surgery,True,Normal,Above Normal,...,Nasal Cannula,True,Not applied,Not applied,Not applied,Not applied,Oral,969,76.0,Death


In [58]:
features_to_keep_005 = ['NutritionIntake', 'Cortisol', 'ProBNP', 'UrineOutput', 'K', 'OxygenTherapy', 'BMI', 'Pulse', 'eGFR', 'TSH', 'FT3', 'Cl', 'Na', 'FT4', 'CRP', 'Ca', 'Mg', 'Ure', 'PCT', 'Lactat', 'Antibiotics', 'Adrenalin', 'Noradrenalin', 'Creatinin', 'PaCO2', 'TroponinI']

In [59]:
matching_rows_diag = matching_rows_diag[features_to_keep_005]

In [60]:
matching_rows_diag

,NutritionIntake,Cortisol,ProBNP,UrineOutput,K,OxygenTherapy,BMI,Pulse,eGFR,TSH,...,Mg,Ure,PCT,Lactat,Antibiotics,Adrenalin,Noradrenalin,Creatinin,PaCO2,TroponinI
22,Oral,Not tested,Not tested,76.0,Below Normal,Nasal Cannula,27.34375,Normal,Below Normal,Not tested,...,Not tested,Normal,Not tested,Above Normal,True,Not applied,Not applied,Above Normal,Not tested,Not tested


In [61]:
matching_rows_diag.dtypes

NutritionIntake     object
Cortisol            object
ProBNP              object
UrineOutput        float64
K                   object
OxygenTherapy       object
BMI                float64
Pulse               object
eGFR                object
TSH                 object
FT3                 object
Cl                  object
Na                  object
FT4                 object
CRP                 object
Ca                  object
Mg                  object
Ure                 object
PCT                 object
Lactat              object
Antibiotics           bool
Adrenalin           object
Noradrenalin        object
Creatinin           object
PaCO2               object
TroponinI           object
dtype: object

In [62]:
model = joblib.load('./model/decision_tree_model.pkl')

In [63]:
model

DecisionTreeClassifier(random_state=42)

In [64]:
scaler = joblib.load('./model/scaler.pkl')

label_encoders = joblib.load('./model/label_encoders.pkl')

In [65]:
print(label_encoders)

{'NutritionIntake': LabelEncoder(), 'Cortisol': LabelEncoder(), 'ProBNP': LabelEncoder(), 'K': LabelEncoder(), 'OxygenTherapy': LabelEncoder(), 'Pulse': LabelEncoder(), 'eGFR': LabelEncoder(), 'TSH': LabelEncoder(), 'FT3': LabelEncoder(), 'Cl': LabelEncoder(), 'Na': LabelEncoder(), 'FT4': LabelEncoder(), 'CRP': LabelEncoder(), 'Ca': LabelEncoder(), 'Mg': LabelEncoder(), 'Ure': LabelEncoder(), 'PCT': LabelEncoder(), 'Lactat': LabelEncoder(), 'Adrenalin': LabelEncoder(), 'Noradrenalin': LabelEncoder(), 'Creatinin': LabelEncoder(), 'PaCO2': LabelEncoder(), 'TroponinI': LabelEncoder(), 'Survival': LabelEncoder()}


In [66]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder

In [67]:
numberical_features = matching_rows_diag.select_dtypes(include=['int64', 'float64']).columns

matching_rows_diag[numberical_features] = scaler.transform(matching_rows_diag[numberical_features])

C:\Users\BonNguyen\AppData\Local\Temp\ipykernel_17656\3329391130.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matching_rows_diag[numberical_features] = scaler.transform(matching_rows_diag[numberical_features])


In [68]:
categorical_cols = matching_rows_diag.select_dtypes(include=['object']).columns

In [69]:
categorical_cols

Index(['NutritionIntake', 'Cortisol', 'ProBNP', 'K', 'OxygenTherapy', 'Pulse',
       'eGFR', 'TSH', 'FT3', 'Cl', 'Na', 'FT4', 'CRP', 'Ca', 'Mg', 'Ure',
       'PCT', 'Lactat', 'Adrenalin', 'Noradrenalin', 'Creatinin', 'PaCO2',
       'TroponinI'],
      dtype='object')

In [70]:
for col in categorical_cols:
    le = label_encoders[col]
    matching_rows_diag[col] = matching_rows_diag[col].apply(lambda x: le.transform([x])[0])

C:\Users\BonNguyen\AppData\Local\Temp\ipykernel_17656\2012922779.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matching_rows_diag[col] = matching_rows_diag[col].apply(lambda x: le.transform([x])[0])
C:\Users\BonNguyen\AppData\Local\Temp\ipykernel_17656\2012922779.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matching_rows_diag[col] = matching_rows_diag[col].apply(lambda x: le.transform([x])[0])
C:\Users\BonNguyen\AppData\Local\Temp\ipykernel_17656\2012922779.py:3: SettingWithCopyWarning: 
A valu

In [71]:
matching_rows_diag

,NutritionIntake,Cortisol,ProBNP,UrineOutput,K,OxygenTherapy,BMI,Pulse,eGFR,TSH,...,Mg,Ure,PCT,Lactat,Antibiotics,Adrenalin,Noradrenalin,Creatinin,PaCO2,TroponinI
22,2,3,2,0.108352,1,2,0.471536,1,1,3,...,3,2,2,0,True,1,1,0,3,2


In [72]:
predict_value = model.predict(matching_rows_diag)
if(predict_value == 0):
    print("alive")
else:
    print("death")

death
